In [ ]:
#optimization routine
N=500
ind=np.linspace(0,len(ideal)-1,N+1,dtype='int')
first=0
last=len(ideal)-1

def naiveOrders(x,first,last,ideal,time):
    #Setup:
    ind=np.zeros((len(x)+2,1)).flatten()
    ind[0]=first
    ind[len(x)+1]=last
    ind[1:len(x)+1]=x
    ind=np.round(ind).astype('int')
    N=len(ind)-1
    print(ind[N])
    controlIdeal=ideal[ind]
    controlTime=time[ind]
    orders=np.zeros((N+1,2))

    #Define orders:
    for i in range(N):
        thisTime = controlTime[i+1]-controlTime[i]
        #speed
        orders[i,0]=(controlIdeal[i+1]-controlIdeal[i])/thisTime
        orders[i,0]=round(orders[i,0]*ieq)
        if orders[i,0] > 14:
            orders[i,0] = 14
        elif orders[i,0] < -14:
            orders[i,0] = -14
        #steps
        if orders[i,0] == 0:
            orders[i,1]=round(thisTime*1000)
            #update control point
            controlIdeal[i+1]=controlIdeal[i]
        else :
            orders[i,1]=abs(orders[i,0]/ieq)*thisTime
            orders[i,1]=round(orders[i,1]/seq)
            #update control point
            controlIdeal[i+1]=controlIdeal[i]+math.copysign(orders[i,1],orders[i,0])*seq
    #Reconstruction:
    rebuilt=np.zeros_like(ideal)
    thisInd=0
    for i in range(orders.shape[0]):
        thisI=orders[i,0]
        thisS=orders[i,1]
        if thisI==0:
            if thisS==0:
                break
            else: #if v==0, s are waiting time in ms
                nextInd= round(thisInd+thisS/1000*framerate).astype('int')
                interval=np.ones((nextInd-thisInd+1,1))*rebuilt[thisInd]
        else:
            thisTime= thisS*seq / (abs(thisI)/ieq)
            #print(thisTime)
            nextInd= thisInd + thisTime*framerate
            nextInd= round(nextInd).astype('int')
            interval=np.linspace(rebuilt[thisInd],
            rebuilt[thisInd]+math.copysign(thisS,thisI)*seq,
            nextInd-thisInd+1)
        
        rebuilt[thisInd:nextInd+1]=interval.flatten()
        thisInd=nextInd
    #Return MSE (error):
    return np.mean(np.power((rebuilt-ideal),2))

""" #x[0] < sin(x[1]) + 1.9
con = lambda x: x[0] - np.sin(x[1])
nlc = opt.NonlinearConstraint(con, -np.inf, 1.9)

const=[]
con=lambda x: x[0]>first
const.append({'type':'ineq','fun':con})
con=lambda x: x[N-1]<last
const.append({'type':'ineq','fun':con})
for i in range(N-2):
    con=lambda x: x[i] - x[i+1]
    #nlc = opt.NonlinearConstraint(con,-np.inf,0)
    const.append({'type':'ineq','fun':con})

res=opt.minimize(
        naiveOrders,
        ind,
        args=(first,last,ideal,time,),
        method='COBYLA',
        tol=1e-6,
        constraints=const
    ) """ #not very interesting to mock a discrete optimization by rounding float variables
    

In [ ]:
def naiveOrders(ind,ideal,time):
    #Setup:
    N=len(ind)-1
    controlIdeal=ideal[ind]
    controlTime=time[ind]
    orders=np.zeros((N+1,2))

    #Define orders:
    for i in range(N):
        thisTime = controlTime[i+1]-controlTime[i]
        #speed
        orders[i,0]=(controlIdeal[i+1]-controlIdeal[i])/thisTime
        orders[i,0]=round(orders[i,0]*ieq)
        if orders[i,0] > 14:
            orders[i,0] = 14
        elif orders[i,0] < -14:
            orders[i,0] = -14
        #steps
        if orders[i,0] == 0:
            orders[i,1]=round(thisTime*1000)
            #update control point
            controlIdeal[i+1]=controlIdeal[i]
        else :
            orders[i,1]=abs(orders[i,0]/ieq)*thisTime
            orders[i,1]=round(orders[i,1]/seq)
            #update control point
            controlIdeal[i+1]=controlIdeal[i]+math.copysign(orders[i,1],orders[i,0])*seq
    #Reconstruction:
    rebuilt=np.zeros_like(ideal)
    thisInd=0
    for i in range(orders.shape[0]):
        thisI=orders[i,0]
        thisS=orders[i,1]
        if thisI==0:
            if thisS==0:
                break
            else: #if v==0, s are waiting time in ms
                nextInd= round(thisInd+thisS/1000*framerate).astype('int')
                interval=np.ones((nextInd-thisInd+1,1))*rebuilt[thisInd]
        else:
            thisTime= thisS*seq / (abs(thisI)/ieq)
            #print(thisTime)
            nextInd= thisInd + thisTime*framerate
            nextInd= round(nextInd).astype('int')
            interval=np.linspace(rebuilt[thisInd],
            rebuilt[thisInd]+math.copysign(thisS,thisI)*seq,
            nextInd-thisInd+1)
        
        rebuilt[thisInd:nextInd+1]=interval.flatten()
        thisInd=nextInd
    #Return MSE (error):
    return np.mean(np.power((rebuilt-ideal),2))

In [ ]:
""" #instead of initializing with a linspace and optimise discreatly, try to select the points based on a ranking and optimise the ranking parameters trough a propper continuous optimization routine

    #0. KEEP IN MIND: initial and final points must be allways selected
    #1. Input a fixed number of control points to select
N=500
    #2. Define the speds of each interval (with no loops)
L=len(ideal)
idealSpeeds = (ideal[1:]-ideal[:L-1])*framerate
    #3. Most interesting points should present great variation of speed (acceleration)
    # BUT, most probably we are not interested (only) in the instantaneous change of speed but in some  sort of longer interval approximation to avoid the noisy patterns. A good scale might be (L/N)*framerate, the mean time different between control points, but lets do it in general and find out.
o = round(L/N)  #order of scale
o=80
oSpeed = (ideal[o:]-ideal[:L-o])*framerate # o=1 recovers instantaneous speed
oInd = np.linspace(int(o/2),L-1-int(o/2),L-o,dtype='int') # index of the ideal vector corresponding
score = abs(oSpeed[1:]-oSpeed[:L-o-1])
scoreInd= oInd[1:]
rank = np.argsort(-score)
rankInd = scoreInd[rank] """

In [ ]:
""" #debug (version 1)
aux=1000
testT=np.linspace(0,100,aux)
test=np.sin(testT/100*4*np.pi)+np.sin(testT/100*7*np.pi)

N=40

L=len(test)

o=8
oSpeed = (test[o:]-test[:L-o]) # o=1 recovers instantaneous speed
oInd = np.linspace(int(o/2),L-1-int(o/2),L-o,dtype='int') # index of the ideal vector corresponding
score = abs(oSpeed[1:]-oSpeed[:L-o-1])
scoreInd= oInd[1:]
rank = np.argsort(-score)

rankInd = scoreInd[rank]

plt.plot(testT,test)
plt.plot(testT[rankInd[:N]],test[rankInd[:N]],linestyle='',marker='o')

iS=test[1:]-test[:L-1]
iSInd=np.linspace(0,L-2,L-1,dtype='int')
iA=iS[1:]-iS[:L-2]
iAInd=np.linspace(0,L-3,L-2,dtype='int')

plt.plot(testT[iSInd],iS*8,color='green')       #speed in green
plt.plot(testT[iAInd],abs(iA*640),color='red')  #acceleration in red
plt.hlines(0,testT[0],testT[len(testT)-1]) """

In [ ]:
""" cleanScore=score.copy()
aux=np.empty(0)
for i in range(len(rankInd)):
#for i in range(N+1):
    if (abs(aux-rank[i])==1).any():
        aux=np.append(aux,rank[i])
        cleanScore[rank[i]]=-10
    else:
        print(i)
        print(aux)
        aux=np.empty(0)
        aux=np.append(aux,rank[i])

cleanRank = np.argsort(-cleanScore)
cleanRankInd = scoreInd[cleanRank]

plt.plot(testT,test)
plt.plot(testT[cleanRankInd[:N]],test[cleanRankInd[:N]],linestyle='',marker='o') """